In [107]:
#% pylab inline
import os
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import glob 
import numpy as np

plt.figure(figsize=(15, 5))

<Figure size 1080x360 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

In [108]:
data, sampling_rate = librosa.load('output10.wav')

In [109]:
#livedf= pd.DataFrame(columns=['feature'])
X, sample_rate = librosa.load('output10.wav', res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
featurelive = mfccs
livedf2 = featurelive

In [110]:
livedf2= pd.DataFrame(data=livedf2)

In [111]:
livedf2 = livedf2.stack().to_frame().T

In [112]:
livedf2

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,-16.76226,-20.33132,-29.128294,-28.858072,-28.955254,-30.632586,-29.637133,-27.938707,-26.989775,-25.306112,...,-26.24641,-31.725262,-30.609581,-30.078857,-28.706011,-30.473549,-32.001976,-33.891747,-34.943394,-37.378765


In [113]:
twodim= np.expand_dims(livedf2, axis=2)

In [114]:
# loading json and creating model
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [115]:
# load weights into new model
loaded_model.load_weights("saved_models/Emotion_Voice_Detection_Model.h5")
print("Loaded model from disk")

Loaded model from disk


In [116]:
livepreds = loaded_model.predict(twodim, 
                         batch_size=32, 
                         verbose=1)

1/1 [==============================] - 0s 997us/step


In [117]:
livepreds

array([[2.8375780e-12, 2.2529315e-31, 1.2428036e-13, 4.6337087e-11,
        5.5774175e-17, 5.3910007e-06, 9.6256757e-01, 3.6589179e-02,
        8.3365460e-04, 4.2636893e-06]], dtype=float32)

In [118]:
livepreds1=livepreds.argmax(axis=1)

In [119]:
liveabc = livepreds1.astype(int).flatten()

In [120]:
liveabc

array([6])

In [121]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
lb.fit(["female_angry","female_calm","female_fearful","female_happy","female_sad","male_angry","male_calm","male_fearful","male_happy","male_sad"])

LabelEncoder()

In [122]:
livepreds

array([[2.8375780e-12, 2.2529315e-31, 1.2428036e-13, 4.6337087e-11,
        5.5774175e-17, 5.3910007e-06, 9.6256757e-01, 3.6589179e-02,
        8.3365460e-04, 4.2636893e-06]], dtype=float32)

In [123]:
livepredictions = (lb.inverse_transform((liveabc)))

In [124]:
livepredictions

array(['male_calm'], dtype='<U14')